In [2]:
from collections import defaultdict
import json

In [12]:
with open('res_cor.json') as data:
    file = json.load(data)

In [13]:
dialogues = [] 
for d in file[:2]:
    samples = defaultdict(dict)
    result = d['completions'][0]['result']
    texts_without_labels = d['data']['text']
    for sample in result:
        speaker = texts_without_labels[int(sample['value']['start'])]['speaker']
        samples[sample['id']]['speaker'] = speaker
        samples[sample['id']]['text'] = sample['value']['text']
        samples[sample['id']]['start'] = int(sample['value']['start'])
        if 'paragraphlabels' in sample['value']:
            samples[sample['id']]['paragraphlabels'] = sample['value']['paragraphlabels'][0]
        if 'choices' in sample['value']:
            samples[sample['id']]['choices'] = sample['value']['choices'][0]
    
    sorted_samples = sorted([(samples[sample_id]['start'], sample_id) for sample_id in samples])
    texts = []
    labels = []
    speakers = []
    for _, sample_id in sorted_samples:
        if samples[sample_id]['text'] != 'PAUSE':
            texts.append(str(samples[sample_id]['text']).replace('\n', ''))
            speakers.append(samples[sample_id]['speaker'])
            paragraph_labels = samples[sample_id].get('paragraphlabels', '')
            choices = samples[sample_id].get('choices', '')
            labels.append(paragraph_labels + '.' + choices)
    dialogues.append((texts, labels, speakers))

In [14]:
train_labels = dialogues[1][1]
test_labels = dialogues[0][1]

In [15]:
class_dict = {}
label_to_name = []
i=0
for el in set(train_labels+test_labels):
    class_dict[el] = i
    i = i+1
    label_to_name.append(el)

In [16]:
A = [[0]*len(class_dict) for _ in range(len(class_dict))]

In [17]:
for label_sequence in (train_labels,test_labels):
    for i,lbl in enumerate(label_sequence):
        if i+1 < len(label_sequence):
            num_class = class_dict[label_sequence[i]]
            num_class2 = class_dict[label_sequence[i+1]]
            A[num_class][num_class2] +=1

In [18]:
for i in range(len(A)):
    total_count=sum(A[i])
    for j in range(len(A[i])):
        A[i][j]/=max(total_count,1)

In [19]:
def print_most_probable_labels(label_name):
    class_id = class_dict[label_name]
    sorted_lbls = sorted(enumerate(A[class_id]),reverse=True,key=lambda x: x[1])[:5]
    for label, probability in sorted_lbls:
        print(label_to_name[label],probability)

In [11]:
print_most_probable_labels('Open.Attend.')

Open.Initiate.Demand.Open.Fact 0.3
Open.Initiate.Give.Fact 0.2
Open.Initiate.Give.Opinion 0.1
React.Rejoinder.Confront.Challenge.Rebound 0.1
Open.Initiate.Demand.Closed.Opinion 0.1
